In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv(".env")

collection_name = "courses_collection"
embedding_dimension = 1536

In [50]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
    https=True
)

openai_client = openai.Client(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [4]:
from qdrant_client.http.models import VectorParams, Distance

collections = qdrant_client.get_collections().collections
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=embedding_dimension,
            distance=Distance.COSINE
        )
    )

In [5]:
from qdrant_client.http.models import PointStruct
import json

with open("recommendation_server/data/processed_courses.json", "r", encoding="utf-8") as f:
    courses = json.load(f)

points = []
for idx, course in enumerate(courses):
    text = f"{course['course_title']}. {course['course_description']}"
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    embedding = response.data[0].embedding
    point = PointStruct(
        id=course['course_code'], #TODO CHANGE TO UUID
        vector=embedding,
        payload=course
    )
    points.append(point)

qdrant_client.upsert(collection_name=collection_name, points=points)

BSN202
BSN203


UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: value BSN202 is not a valid point ID, valid values are either an unsigned integer or a UUID"},"time":0.0}'

In [46]:
from qdrant_client.http import models

def recommend_courses(query_text: str, top_k: int = 3):
    search_response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query_text
    )
    query_embedding = search_response.data[0].embedding
    search_results = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k,
        query_filter=None
    )

    print(search_results)

    recommendations = []
    for result in search_results.points:
        recommendations.append({
            "id": result.id,
            "title": result.payload.get("title"),
            "description": result.payload.get("description"),
            "score": result.score
        })
    return recommendations


In [47]:
recommend_courses("Im looking for courses that will help me learn backend", top_k=3)

points=[ScoredPoint(id=4, version=0, score=0.4949369, payload={'title': 'Web Development Bootcamp', 'description': 'An all-in-one course covering both front-end and back-end web development. Students learn HTML, CSS, JavaScript, and server-side programming with modern frameworks through real-world projects and website deployments.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=8, version=0, score=0.41414928, payload={'title': 'Advanced Java Programming', 'description': 'This course covers advanced Java topics such as concurrency, memory management, and design patterns. Through in-depth projects, learners develop robust applications and gain insights into best practices in enterprise-level Java development.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=9, version=0, score=0.40244797, payload={'title': 'Cloud Computing Essentials', 'description': 'An introductory course to cloud computing concepts, services, and deployment models. The curriculum foc

[{'id': 4,
  'title': 'Web Development Bootcamp',
  'description': 'An all-in-one course covering both front-end and back-end web development. Students learn HTML, CSS, JavaScript, and server-side programming with modern frameworks through real-world projects and website deployments.',
  'score': 0.4949369},
 {'id': 8,
  'title': 'Advanced Java Programming',
  'description': 'This course covers advanced Java topics such as concurrency, memory management, and design patterns. Through in-depth projects, learners develop robust applications and gain insights into best practices in enterprise-level Java development.',
  'score': 0.41414928},
 {'id': 9,
  'title': 'Cloud Computing Essentials',
  'description': 'An introductory course to cloud computing concepts, services, and deployment models. The curriculum focuses on major platforms such as AWS, Azure, and Google Cloud, and includes hands-on labs to build and deploy cloud-based applications.',
  'score': 0.40244797}]